# Predicción con nuestro clasificador custom de Comprehend

In [3]:
import boto3
import sagemaker
import pandas as pd

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

from botocore.config import Config

config = Config(retries={"max_attempts": 10, "mode": "adaptive"})

comprehend = boto3.Session().client(service_name="comprehend", region_name=region)

In [4]:
%store -r comprehend_training_job_arn

In [5]:
try:
    comprehend_training_job_arn
except NameError:
    print("***************************************************************************")
    print("[ERROR] PLEASE WAIT FOR THE PREVIOUS NOTEBOOK TO FINISH *******************")
    print("[ERROR] OR THIS NOTEBOOK WILL NOT RUN PROPERLY ****************************")
    print("***************************************************************************")

In [6]:
print(comprehend_training_job_arn)

arn:aws:comprehend:us-east-1:688013747199:document-classifier/Amazon-Customer-Reviews-Classifier-1668541045


In [7]:
%store -r comprehend_endpoint_arn

In [8]:
try:
    comprehend_endpoint_arn
except NameError:
    print("***************************************************************************")
    print("[ERROR] PLEASE WAIT FOR THE PREVIOUS NOTEBOOK TO FINISH *******************")
    print("[ERROR] OR THIS NOTEBOOK WILL NOT RUN PROPERLY ****************************")
    print("***************************************************************************")

In [9]:
print(comprehend_endpoint_arn)

arn:aws:comprehend:us-east-1:688013747199:document-classifier-endpoint/comprehend-inference-ep-15-20-28-20


## Desplegamos el endpoint

In [10]:
describe_response = comprehend.describe_endpoint(EndpointArn=comprehend_endpoint_arn)
print(describe_response)

{'EndpointProperties': {'EndpointArn': 'arn:aws:comprehend:us-east-1:688013747199:document-classifier-endpoint/comprehend-inference-ep-15-20-28-20', 'Status': 'CREATING', 'ModelArn': 'arn:aws:comprehend:us-east-1:688013747199:document-classifier/Amazon-Customer-Reviews-Classifier-1668541045', 'DesiredModelArn': 'arn:aws:comprehend:us-east-1:688013747199:document-classifier/Amazon-Customer-Reviews-Classifier-1668541045', 'DesiredInferenceUnits': 1, 'CurrentInferenceUnits': 0, 'CreationTime': datetime.datetime(2022, 11, 15, 20, 28, 20, 452000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2022, 11, 15, 20, 28, 20, 452000, tzinfo=tzlocal())}, 'ResponseMetadata': {'RequestId': 'be628745-5624-42d8-87f7-b3ee1382947e', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'be628745-5624-42d8-87f7-b3ee1382947e', 'content-type': 'application/x-amz-json-1.1', 'content-length': '536', 'date': 'Tue, 15 Nov 2022 20:29:41 GMT'}, 'RetryAttempts': 0}}


## Comprobamos el estado el endpoint

In [12]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/comprehend/v2/home?region={}#classifier-details/{}/endpoints/{}/details">Comprehend Model Endpoint</a></b>'.format(
            region, comprehend_training_job_arn, comprehend_endpoint_arn
        )
    )
)

## Esta celda puede tardar más de ----- minutos en ejecutarse.

In [13]:
%%time

import time

max_time = time.time() + 3 * 60 * 60  # 3 hours
while time.time() < max_time:
    describe_response = comprehend.describe_endpoint(EndpointArn=comprehend_endpoint_arn)
    status = describe_response["EndpointProperties"]["Status"]
    print("Endpoint: {}".format(status))

    if status == "IN_SERVICE" or status == "IN_ERROR":
        break

    time.sleep(5)

Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CR

# [INFO] _Feel free to continue to the next workshop section while this notebook is running._

## Enviamos una predicción al endpoint

In [14]:
txt = """I loved it!  I will recommend this to everyone."""

response = comprehend.classify_document(Text=txt, EndpointArn=comprehend_endpoint_arn)

import json

print(json.dumps(response, indent=2, default=str))

{
  "Classes": [
    {
      "Name": "1",
      "Score": 0.5084999799728394
    },
    {
      "Name": "5",
      "Score": 0.48690545558929443
    },
    {
      "Name": "2",
      "Score": 0.004616440273821354
    }
  ],
  "ResponseMetadata": {
    "RequestId": "31b6b5e3-5d4a-4964-bd0c-088e2d71dc06",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "x-amzn-requestid": "31b6b5e3-5d4a-4964-bd0c-088e2d71dc06",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "136",
      "date": "Tue, 15 Nov 2022 20:51:27 GMT"
    },
    "RetryAttempts": 0
  }
}


In [15]:
txt = """It's OK."""

response = comprehend.classify_document(Text=txt, EndpointArn=comprehend_endpoint_arn)

import json

print(json.dumps(response, indent=2, default=str))

{
  "Classes": [
    {
      "Name": "5",
      "Score": 0.9999997615814209
    },
    {
      "Name": "4",
      "Score": 2.0272238543839194e-05
    },
    {
      "Name": "2",
      "Score": 1.0010059668275062e-05
    }
  ],
  "ResponseMetadata": {
    "RequestId": "c7dcb0c7-f490-499f-9963-34cd7a0f5a24",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "x-amzn-requestid": "c7dcb0c7-f490-499f-9963-34cd7a0f5a24",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "139",
      "date": "Tue, 15 Nov 2022 20:51:31 GMT"
    },
    "RetryAttempts": 0
  }
}


In [16]:
txt = """Really bad.  I hope they don't make this anymore."""

response = comprehend.classify_document(Text=txt, EndpointArn=comprehend_endpoint_arn)

import json

print(json.dumps(response, indent=2, default=str))

{
  "Classes": [
    {
      "Name": "1",
      "Score": 0.8193185925483704
    },
    {
      "Name": "2",
      "Score": 0.17978569865226746
    },
    {
      "Name": "5",
      "Score": 0.0007932941080071032
    }
  ],
  "ResponseMetadata": {
    "RequestId": "5eb4892d-65ac-4c04-a9d3-e4df3c60b886",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "x-amzn-requestid": "5eb4892d-65ac-4c04-a9d3-e4df3c60b886",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "136",
      "date": "Tue, 15 Nov 2022 20:51:33 GMT"
    },
    "RetryAttempts": 0
  }
}


# Release Resources

In [17]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}

<IPython.core.display.Javascript object>

In [18]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>